In [23]:
import tweepy
import pandas as pd
import time

# Bearer tokens
BEARER_TOKENS = [
    "AAAAAAAAAAAAAAAAAAAAAF0sxQEAAAAARrIIO7syGsyteGGoMUBoU1zL0XI%3DyJC0Vj1RvRmcGP39zjB4cfkYONcC5ouFrHwXiBrcQufdOT0QDH",
    "AAAAAAAAAAAAAAAAAAAAABNfxQEAAAAAlPtf0kffuoJchGZEjJrHsELAILU%3DxaUkeKwsCfl4wwVxVb8QKezihitPsxbsk2FDIgG4bBibRQdPiw",
    "AAAAAAAAAAAAAAAAAAAAADcsxQEAAAAA1FJXhrBL%2FXo3Nlw78YC5TVO4jns%3D30eM1mRWzb9erLFlvSrqupvdS9pByR6xE530Vn6gJmK26ychGb"
]

# Queries
queries = [
        '(Conflict Kosovo OR Conflict Serbia OR RKS OR SRB) lang:en -is:retweet'
]

# Parameters for tweet collection
max_tweets_per_query = 100
tweets_per_request = 50
output_file = "fortesting.csv"

# Function to save data to a CSV file
def save_to_csv(data, filename):
    df = pd.DataFrame(data, columns=["Query", "Tweet ID", "Author ID", "Created At", "Text"])
    header = not pd.io.common.file_exists(filename)
    df.to_csv(filename, mode='a', header=header, index=False)
    print(f"Saved {len(data)} tweets to {filename}")

# Function to fetch tweets
def fetch_tweets(client, query):
    tweets_data = []
    tweet_ids = set()
    tweet_count = 0
    next_token = None
    print("Fetching tweets count... ", tweet_count)
    print("Fetching tweets count... ", tweet_count)

    while tweet_count < max_tweets_per_query:
        try:
            response = client.search_recent_tweets(
                query=query,
                max_results=tweets_per_request,
                next_token=next_token,
                tweet_fields=["created_at", "author_id", "id"],
            )

            if response.data:
                for tweet in response.data:
                    if tweet.id not in tweet_ids:
                        tweet_ids.add(tweet.id)
                        tweets_data.append([query, tweet.id, tweet.author_id, tweet.created_at, tweet.text])
                        tweet_count += 1

            # Save tweets in batches
            if tweets_data:
                save_to_csv(tweets_data, output_file)
                tweets_data = []

            # Check for next_token
            next_token = response.meta.get('next_token') if response.meta else None
            if not next_token or tweet_count >= max_tweets_per_query:
                break

            time.sleep(2)  # Short delay between requests

        except tweepy.TooManyRequests:
            print(tweepy.TweepyException)
            print("Rate limit exceeded, waiting for 15 minutes...")
            time.sleep(15 * 60)
        except Exception as e:
            print(f"Error fetching tweets: {e}")
            break

# Main script
if __name__ == "__main__":
    for bearer_token in BEARER_TOKENS:
        client = tweepy.Client(bearer_token=bearer_token)
        print(f"Using bearer token: {bearer_token[:20]}...")

        for query in queries:
            print(f"Executing query: {query}")
            fetch_tweets(client, query)

        print("Switching to the next bearer token...")
        time.sleep(15 * 60)  # Ensure no overlap in rate limits

    print("Tweet collection complete.")


Using bearer token: AAAAAAAAAAAAAAAAAAAA...
Executing query: (Conflict Kosovo OR Conflict Serbia OR RKS OR SRB) lang:en -is:retweet
Fetching tweets count...  0
Fetching tweets count...  0
<class 'tweepy.errors.TweepyException'>
Rate limit exceeded, waiting for 15 minutes...


KeyboardInterrupt: 